In [1]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import ipyparallel as ipp
import pandas as pd
import numpy as np
import os
import seaborn as sns
from IPython.display import display
import csv

from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import make_regression

import MESS
from MESS.util import set_params

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## set directory for analysis
analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"

##make if doesn't exist
if not os.path.exists(analysis_dir):
    os.mkdir(analysis_dir)

/mnt/lfs2/ruff6699/Mess2.0/git/MESS/MESS/__init__.py:21: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/mnt/ceph/ruff6699/scratch/Mess2.0/minicoiconda2/envs/py2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/mnt/ceph/ruff6699/scratch/Mess2.0/minicoiconda2/envs/py2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/mnt/lfs2/ruff6699/Mess2.0/minicoiconda2/envs/py2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/mnt/ceph/ruff6699/scratch/Mess2.0/minicoiconda2/envs/py2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in

In [4]:
## quick test to make sure Mess will run
r = MESS.Region("r1")
r.paramsdict["generations"] = 0.25
r.set_param("project_dir", analysis_dir)
r.set_param("m", 0.01)
r.run(sims=1)

  Project directory exists. Additional simulations will be appended.
    Generating 1 simulation(s).
  [####################] 100%  Finished 0 simulations    | 0:00:00 | 
 

In [ ]:
## run the below line in terminal, not in jupyter notebooks
## this will start and ipcluster 
## ipcluster start -n 40 --cluster-id="MESS-Rich" --daemonize

In [5]:
## check to make sure the client is working
## should print the number of cores ready to go
ipyclient = ipp.Client(cluster_id="MESS-Rich")
print(len(ipyclient))

39


In [ ]:
## This chunk runs a bunch of simulations with different param combos

## set dir (again) bc just in case..
analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"

## different param combinations to try
specrates = np.array([0.0, 0.0001, 0.001])
modnames = np.array(["neutral", "filtering", "competition"])
localcomsize = np.array([500, 1000, 2000])

## go through all param combinations
params = [specrates, modnames, localcomsize]
params = list(itertools.product(*params))
for i, p in enumerate(params):
    specrates, modnames, localcomsize = p
    print(specrates, modnames, localcomsize)
    ldir = analysis_dir + "Speciation-{}/".format(specrates)
    if not os.path.exists(ldir):
        os.mkdir(ldir)
    ldir = ldir + "LocalComSize-{}".format(localcomsize)
    if not os.path.exists(ldir):
        os.mkdir(ldir)
    r = MESS.Region("sim-{}".format(i))
    r._log_files = True
    
    r.set_param("generations", 0.5)
    r.set_param("community_assembly_model", modnames)
    r.set_param("S_m", 500)
    r.set_param("J", localcomsize)
    r.set_param("speciation_rate", specrates)
    r.set_param("project_dir", ldir)

    r.run(sims=1000, ipyclient=ipyclient, quiet=False)
    #r.run(sims=100, quiet=False)
    

(0.0, 'neutral', 500)
  Updating Metacommunity parameters requires running set_metacommunity() to apply the changes.
  Project directory exists. Additional simulations will be appended.
    Generating 1000 simulation(s).
  [####################] 100%  Performing Simulations    | 0:01:53 | 
  [####################] 100% 
    Finished 1000 simulations
 (0.0, 'neutral', 1000)
  Updating Metacommunity parameters requires running set_metacommunity() to apply the changes.
  Project directory exists. Additional simulations will be appended.
    Generating 1000 simulation(s).
  [####################] 100%  Performing Simulations    | 0:05:42 | 
  [####################] 100% 
    Finished 1000 simulations
 (0.0, 'neutral', 2000)
  Updating Metacommunity parameters requires running set_metacommunity() to apply the changes.
  Project directory exists. Additional simulations will be appended.
    Generating 1000 simulation(s).
  [####################] 100%  Performing Simulations    | 0:18:37 | 
 

In [28]:
## This whole chunk is like data exploration

analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0"
##load data for one plot

ldir = analysis_dir + "/LocalComSize-500/SIMOUT.txt"
##Begin with No Speciation
df = pd.read_csv(ldir, sep="\t", header=0)

#print(loccom.columns)

#print(loccom.loc[:, "S":"abund_h4"])
#print(loccom.loc[:, "pi_h1":"iqr_dxys"])
#print(loccom.loc[:, "SGD_0":"SGD_9"])
#print(loccom.loc[:, "mean_local_traits":"reg_loc_iqr_trait_dif"])

genetic_flag = True
abund_flag = False
trait_flag = True


genX = pd.concat([df.loc[:, "pi_h1":"iqr_dxys"], df.loc[:, "SGD_0":"SGD_9"]], axis=1)
abundX = df.loc[:, "S":"abund_h4"]  
traitX = df.loc[:, "mean_local_traits":"reg_loc_iqr_trait_dif"] 

       ## y will always be this
y = df['community_assembly_model']
    
X = df['community_assembly_model']
if (genetic_flag == True):
    X = pd.concat([X, genX], axis=1)
elif (abund_flag == True):
    X = pd.concat([X, abundX], axis=1)
elif (trait_flag == True):  
    X = pd.concat([X, traitX], axis=1)

X = X.drop(['community_assembly_model'], axis=1)
X

,pi_h1,pi_h2,pi_h3,pi_h4,mean_pi,std_pi,skewness_pi,kurtosis_pi,median_pi,iqr_pi,mean_dxys,std_dxys,skewness_dxys,kurtosis_dxys,median_dxys,iqr_dxys,SGD_0,SGD_1,SGD_2,SGD_3,SGD_4,SGD_5,SGD_6,SGD_7,SGD_8,SGD_9
0,2.796571,2.651376,2.550938,2.480410,0.000124,0.000279,2.056966,2.690958,0.000000,0.000000,0.001020,0.001101,1.051648,-0.061627,0.000702,0.001272,13.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,4.752226,3.521397,2.985570,2.728739,0.000392,0.000738,2.561786,5.837293,0.000000,0.000351,0.001579,0.001690,1.041567,0.067516,0.000702,0.002281,9.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,3.987807,3.455558,3.183188,3.027976,0.000273,0.000579,2.214917,3.648801,0.000000,0.000175,0.000840,0.001290,1.815727,2.713588,0.000175,0.001316,14.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,4.826410,4.651163,4.483358,4.330632,0.000260,0.000440,1.338725,0.344766,0.000000,0.000526,0.001199,0.001212,0.691247,-0.626316,0.000614,0.001930,13.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0
4,6.632732,6.297464,6.013299,5.784444,0.000397,0.000493,0.762170,-0.856422,0.000000,0.000673,0.001272,0.001790,1.967814,3.325758,0.000526,0.001754,9.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,2.0
5,3.856988,3.705411,3.557289,3.423307,0.000105,0.000191,1.534813,1.004494,0.000000,0.000088,0.000800,0.000979,1.643700,2.212247,0.000439,0.001009,12.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0
6,6.299853,5.878261,5.624941,5.463390,0.000358,0.000492,0.944481,-0.781020,0.000000,0.000702,0.001311,0.001721,1.609853,1.823030,0.000702,0.001754,10.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0
7,5.225238,4.760151,4.471861,4.275327,0.000395,0.000551,1.101164,-0.198541,0.000000,0.000789,0.001241,0.001455,1.695557,2.320012,0.000702,0.001360,8.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0
8,3.940566,3.878187,3.814614,3.751783,0.000180,0.000319,1.305842,-0.037385,0.000000,0.000156,0.000822,0.001114,1.746248,2.622374,0.000439,0.001447,12.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0
9,2.295491,1.918537,1.742995,1.656109,0.000171,0.000495,3.317197,9.961197,0.000000,0.000000,0.001296,0.001673,1.239862,0.425176,0.000263,0.001886,15.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
## Here, estimate the best RF parameters to make a classifier that has the most power. 
## also stole this chunk from Isaac

## We will be using this data to tune the classification parameters
ldir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt"
df = pd.read_csv(ldir, sep="\t", header=0)

## asign data
y = df['community_assembly_model']
X = df.loc[:, "S":"SGD_9"]

## split data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

## get best RF params
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

## Randomly search 100 different parameter combinations and take the
# ## one that reduces CV error
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(),\
                               param_distributions = random_grid,
                               n_iter = 100, cv = 3, verbose=0, n_jobs = -1, 
                               error_score=np.nan)

# ## fit the training data using best params for RF
#rf_random.fit(Xtrain, ytrain)

rf_random.fit(Xtrain, ytrain)
    
ypred = rf_random.predict(Xtest)
cm = metrics.confusion_matrix(ypred, ytest)
print(cm)

[[224  36   7]
 [ 25 212   3]
 [  2   9 231]]


In [4]:
#print(cm)
print(rf_random.best_params_["n_estimators"])

ldir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt"
df = pd.read_csv(ldir, sep="\t", header=0)

## asign data
y = df['community_assembly_model']
X = df.loc[:, "S":"SGD_9"]

## split data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)  

model = RandomForestClassifier(n_estimators=rf_random.best_params_["n_estimators"], \
                               n_jobs=-1, max_features='auto', max_depth=rf_random.best_params_["max_depth"], \
                               bootstrap=rf_random.best_params_["bootstrap"], \
                               min_samples_split=rf_random.best_params_["min_samples_split"], \
                               min_samples_leaf=rf_random.best_params_["min_samples_leaf"])

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)
cm = metrics.confusion_matrix(ypred, ytest)
print(cm)


classify_model = RF_classify(df, rf_random)
print(classify_model)

#print(rf_random.cv_results_)

NameError: name 'rf_random' is not defined

In [21]:
## This chunk is the function to perform RF and cross validation to asses
## the error rates for each model

def RF_classify(df, rf_random, genetic_flag = True, abund_flag = True, trait_flag = True):
    
    #print(rf_random.best_params_)
   
    ## split up the sumstats into their type, join later if all flags == True
    genX = pd.concat([df.loc[:, "pi_h1":"iqr_dxys"], df.loc[:, "SGD_0":"SGD_9"]], axis=1)
    #genX = df.loc[:, "pi_h1":"iqr_dxys"]
    abundX = df.loc[:, "S":"abund_h4"]  
    traitX = df.loc[:, "mean_local_traits":"reg_loc_iqr_trait_dif"] 

    ## y will always be this
    y = df['community_assembly_model']
    
    ## begin X as this
    X = df['community_assembly_model']
    
    ## add certain sumstats depending on which flag is on
    if (genetic_flag == True):
        X = pd.concat([X, genX], axis=1)
    if (abund_flag == True):
        X = pd.concat([X, abundX], axis=1)
    if (trait_flag == True):  
        X = pd.concat([X, traitX], axis=1)

    ## drop that first column, you don't need it bc the info is in y
    X = X.drop(['community_assembly_model'], axis=1)

    #y = df['community_assembly_model']
    #X = df.loc[:, "S":"SGD_9"]

    # split data
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)  

    model = RandomForestClassifier(n_estimators=rf_random.best_params_["n_estimators"], \
                                   n_jobs=-1, max_features='auto', max_depth=rf_random.best_params_["max_depth"], \
                                   bootstrap=rf_random.best_params_["bootstrap"], \
                                   min_samples_split=rf_random.best_params_["min_samples_split"], \
                                   min_samples_leaf=rf_random.best_params_["min_samples_leaf"])

    model.fit(Xtrain, ytrain)
    ypred = model.predict(Xtest)
    cm = metrics.confusion_matrix(ypred, ytest)
    #print(cm)

    err = []
    for i in range(0,3):
        #competition first, then filtering, then neutral, then average 
        acuracy = float(cm[i][i])
        total = float(sum([cm[0][i], cm[1][i], cm[2][i]]))
        z = int(total)-int(acuracy)
        Accuracy_array = np.concatenate([np.ones(int(acuracy)), np.zeros(z)])
        
        print(np.mean(Accuracy_array))
        print(np.std(Accuracy_array))
        
        err.append(acuracy/total)
        
    #append avg error rate from our own external validation
    err.append(np.mean(err))    
    
#     #also append average error rate from cv function
#     scores = cross_val_score(model, Xtrain, ytrain, cv=5)
#     err.append(scores.mean()) 

    return err

In [10]:
## ! means run in the shell
## lists the number of lines in the $filename
!wc -l $ldir 

2991 /mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-2000/SIMOUT.txt


In [22]:
analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001"

ldir = analysis_dir + "/LocalComSize-1000/SIMOUT.txt"
print(ldir)
##Begin with No Speciation
loccom_500_df = pd.read_csv(ldir, sep="\t", header=0)
#print(loccom_500_df)
#print(loccom_500_df["community_assembly_model"])
print(len(loccom_500_df))
classify_model = RF_classify(loccom_500_df, rf_random)
print(classify_model)

/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt
2993
0.8438818565400844
0.3629673108456804
0.8764478764478765
0.32906989579401574
0.9565217391304348
0.20393111999232305
[0.8438818565400844, 0.8764478764478765, 0.9565217391304348, 0.8922838240394654]


In [89]:
## In this chunk, go through and run RF classifier on all data

analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"
allErrorRates = []

specrates = np.array([0.0, 0.0001])
modnames = np.array(["neutral", "filtering", "competition"])
localcomsize = np.array([500, 1000, 2000])

params = [specrates, localcomsize]
params = list(itertools.product(*params))

for i, p in enumerate(params):
    specrates, localcomsize = p
    #print(specrates, localcomsize)
    ldir = analysis_dir + "Speciation-{}/".format(specrates)
    ldir = ldir + "LocalComSize-{}/".format(localcomsize)
    ldir = ldir + "SIMOUT.txt"
    print(ldir)
    df = pd.read_csv(ldir, sep="\t", header=0)
    classify_model = RF_classify(df, rf_random)
    print(classify_model)
    output = [specrates, localcomsize] + classify_model
    allErrorRates.append(output)

print(allErrorRates)
csvData = allErrorRates
with open('/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/allErrorRates.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()


/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-500/SIMOUT.txt
[0.8435114503816794, 0.7590361445783133, 0.8818565400843882, 0.8281347116814604]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-1000/SIMOUT.txt
[0.8846153846153846, 0.8604651162790697, 0.9571984435797666, 0.9007596481580736]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-2000/SIMOUT.txt
[0.8870292887029289, 0.9057377049180327, 0.9700374531835206, 0.9209348156014941]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-500/SIMOUT.txt
[0.8582677165354331, 0.7215686274509804, 0.9163179916317992, 0.8320514452060709]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt
[0.8888888888888888, 0.8450184501845018, 0.9426229508196722, 0.8921767632976877]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-2000/SIMOUT.txt
[0.8917748917748918, 0.8

In [90]:
## In this chunk, go through and run RF classifier with only Genetic and Abundance stats

analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"

GA_ErrorRates = []

specrates = np.array([0.0, 0.0001])
modnames = np.array(["neutral", "filtering", "competition"])
localcomsize = np.array([500, 1000, 2000])

params = [specrates, localcomsize]
params = list(itertools.product(*params))

for i, p in enumerate(params):
    specrates, localcomsize = p
    #print(specrates, localcomsize)
    ldir = analysis_dir + "Speciation-{}/".format(specrates)
    ldir = ldir + "LocalComSize-{}/".format(localcomsize)
    ldir = ldir + "SIMOUT.txt"
    print(ldir)
    df = pd.read_csv(ldir, sep="\t", header=0)
    
    ## trait flag on False
    classify_model = RF_classify(df, rf_random, trait_flag=False)
    print(classify_model)
    output = [specrates, localcomsize] + classify_model
    GA_ErrorRates.append(output)

    
print(GA_ErrorRates)
csvData = GA_ErrorRates
with open('/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/GA_ErrorRates.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()

/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-500/SIMOUT.txt
[0.5409836065573771, 0.490272373540856, 0.9149797570850202, 0.648745245727751]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-1000/SIMOUT.txt
[0.5795918367346938, 0.5955056179775281, 0.9367088607594937, 0.7039354384905718]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-2000/SIMOUT.txt
[0.5975609756097561, 0.6823529411764706, 0.9598393574297188, 0.7465844247386485]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-500/SIMOUT.txt
[0.5241635687732342, 0.5604838709677419, 0.8614718614718615, 0.6487064337376125]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt
[0.5776892430278885, 0.5311203319502075, 0.9727626459143969, 0.6938574069641642]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-2000/SIMOUT.txt
[0.5757575757575758, 0.678

In [91]:
## In this chunk, go through and run RF classifier with only Genetic and Trait stats

analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"

GT_ErrorRates = []

specrates = np.array([0.0, 0.0001])
modnames = np.array(["neutral", "filtering", "competition"])
localcomsize = np.array([500, 1000, 2000])

params = [specrates, localcomsize]
params = list(itertools.product(*params))

for i, p in enumerate(params):
    specrates, localcomsize = p
    #print(specrates, localcomsize)
    ldir = analysis_dir + "Speciation-{}/".format(specrates)
    ldir = ldir + "LocalComSize-{}/".format(localcomsize)
    ldir = ldir + "SIMOUT.txt"
    print(ldir)
    df = pd.read_csv(ldir, sep="\t", header=0)
    
    ## trait flag on False
    classify_model = RF_classify(df, rf_random, abund_flag=False, trait_flag=True)
    print(classify_model)
    output = [specrates, localcomsize] + classify_model
    GT_ErrorRates.append(output)

    
print(GT_ErrorRates)
csvData = GT_ErrorRates
with open('/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/GT_ErrorRates.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()

/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-500/SIMOUT.txt
[0.803347280334728, 0.7859922178988327, 0.8928571428571429, 0.8273988803635679]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-1000/SIMOUT.txt
[0.8795180722891566, 0.83203125, 0.9672131147540983, 0.8929208123477516]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-2000/SIMOUT.txt
[0.8689138576779026, 0.8775510204081632, 0.9621848739495799, 0.9028832506785486]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-500/SIMOUT.txt
[0.825, 0.7340823970037453, 0.8713692946058091, 0.8101505638698514]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt
[0.8674698795180723, 0.7992565055762082, 0.9134199134199135, 0.8600487661713979]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-2000/SIMOUT.txt
[0.9054054054054054, 0.8590308370044053, 0.979

In [92]:
## In this chunk, go through and run RF classifier with only Genetic and Abundance stats

analysis_dir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/"

AT_ErrorRates = []

specrates = np.array([0.0, 0.0001])
modnames = np.array(["neutral", "filtering", "competition"])
localcomsize = np.array([500, 1000, 2000])

params = [specrates, localcomsize]
params = list(itertools.product(*params))

for i, p in enumerate(params):
    specrates, localcomsize = p
    #print(specrates, localcomsize)
    ldir = analysis_dir + "Speciation-{}/".format(specrates)
    ldir = ldir + "LocalComSize-{}/".format(localcomsize)
    ldir = ldir + "SIMOUT.txt"
    print(ldir)
    df = pd.read_csv(ldir, sep="\t", header=0)
    
    ## trait flag on False
    classify_model = RF_classify(df, rf_random, genetic_flag=False, abund_flag=True, trait_flag=True)
    print(classify_model)
    output = [specrates, localcomsize] + classify_model
    AT_ErrorRates.append(output)

    
print(AT_ErrorRates)
csvData = AT_ErrorRates
with open('/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/AT_ErrorRates.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()

/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-500/SIMOUT.txt
[0.8352490421455939, 0.8163265306122449, 0.8471074380165289, 0.8328943369247893]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-1000/SIMOUT.txt
[0.8643410852713178, 0.84251968503937, 0.9493670886075949, 0.8854092863060942]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0/LocalComSize-2000/SIMOUT.txt
[0.888030888030888, 0.9109311740890689, 0.9549180327868853, 0.917960031635614]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-500/SIMOUT.txt
[0.8307692307692308, 0.7398373983739838, 0.8801652892561983, 0.8169239727998043]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt
[0.8864468864468864, 0.8986784140969163, 0.9397590361445783, 0.908294778896127]
/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-2000/SIMOUT.txt
[0.8648648648648649, 0.916666

In [ ]:
##stole from Isaac
features = sim_df.iloc[:, 22:].columns

## Parameters to estimate
targets = ["alpha", "J_m", "ecological_strength", "m", "speciation_prob", "_lambda"]

X = sim_df[features]
y = sim_df[targets]

## Split the data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
display(Xtrain[:5])
display(ytrain[:5])

In [6]:
## also stole this chunk from ISaac
ldir = "/mnt/lfs2/ruff6699/Mess2.0/git/ModelPerformance_2/Speciation-0.0001/LocalComSize-1000/SIMOUT.txt"
df = pd.read_csv(ldir, sep="\t", header=0)
print(len(df))

## asign data
y = df['community_assembly_model']
X = df.loc[:, "S":"SGD_9"]

## split data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

## get best RF params
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

## Randomly search 100 different parameter combinations and take the
# ## one that reduces CV error
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(),\
                               param_distributions = random_grid,
                               n_iter = 100, cv = 3, verbose=0, n_jobs = -1, 
                               error_score=np.nan)

# ## fit the training data using best params for RF
rf_random.fit(Xtrain, ytrain)


2993


RandomizedSearchCV(cv=3, error_score=nan,
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'bootstrap': [True, False], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_leaf': [1, 2, 4]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [10]:
model = rf_random.best_estimator_
ypred = model.predict(Xtest)
cm = metrics.confusion_matrix(ypred, ytest)
print(cm)

err = []
for i in range(0,3):
    #competition first, then filtering, then neutral, then average 
    acuracy = float(cm[i][i])
    total = float(sum([cm[0][i], cm[1][i], cm[2][i]]))
    err.append(acuracy/total)
    
print(err)

#ypred = model.predict(Xtest)
# print(ytest)
# print(ypred)

# cm = metrics.confusion_matrix(ypred, ytest)


## Split the data
# Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

# model = RandomForestClassifier(n_estimators=100, n_jobs=-1, min_samples_split=5, min_samples_leaf=4,\
#                               max_features='auto', max_depth=None, bootstrap=True)
# model.fit(Xtrain, ytrain)
# ypred = model.predict(Xtest)

# cm = metrics.confusion_matrix(ypred, ytest)

# err = []
# for i in range(0,3):
#     #competition first, then filtering, then neutral, then average 
#     acuracy = float(cm[i][i])
#     total = float(sum([cm[0][i], cm[1][i], cm[2][i]]))
#     err.append(acuracy/total)
    
# #append avg error rate from our own external validation
# err.append(np.mean(err))    

# #also append average error rate from cv function
# scores = cross_val_score(model, Xtrain, ytrain, cv=5)
# err.append(scores.mean()) 


[[213  34   6]
 [ 30 216   9]
 [  3   6 232]]
[0.8658536585365854, 0.84375, 0.9392712550607287]


In [ ]:
##plot example store from isaac

lims = {"m":0.01, "speciation_prob":0.001}
for i, p in enumerate(targets):
    fig, ax = plt.subplots()
    vscore = metrics.explained_variance_score(y.iloc[:, i], cv_preds[:, i])
    r2 = metrics.r2_score(y.iloc[:, i], cv_preds[:, i])
    print(p, vscore, r2)
    ax.scatter(y.iloc[:, i], cv_preds[:, i], c='black', marker='.', s=2)
    ax.set_title(p)
    if p in ["m", "speciation_prob"]:
        ax.set_xlim(0, lims[p])
        ax.set_ylim(0, lims[p])